In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 7.5 MB/s 
     |████████████████████████████████| 60 kB 5.6 MB/s 


In [ ]:
# La clase `Model` se hace cargo de los atributos a nivel del modelo, maneja los agentes. 
# Cada modelo puede contener múltiples agentes y todos ellos son instancias de la clase `Agent`.
from mesa import Agent, Model 

# Con `SimultaneousActivation` hacemos que todos los agentes se activen de manera simultanea.
from mesa.time import SimultaneousActivation

# Vamos a hacer uso de `DataCollector` para obtener el grid completo cada paso (o generación) y lo usaremos para graficarlo.
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Definimos los siguientes paquetes para manejar valores númericos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime


#Definición del grid


In [ ]:
def get_grid(model):


    #NOTA IMPORTANTE DEL GRID. como ahora no estamos haciendo posiciones al azar, todo empezara en posiciones predeterminadas, por lo que podemos poner cada semaforo individualmente, asi como puntos pivote donde
    # SI el carro va a girar, y llega a cierto X,Y = haga el giro.

    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for content in cell_content:
          if isinstance(content, Carro):
            grid[x][y] = 5
          else:
            grid[x][y] = content.live
        
    return grid

In [ ]:
time_end = time.time() + 20

class Semaforo(Agent):

    def __init__(self, unique_id, pos, model, isFirst, otherX, otherY):

        super().__init__(unique_id, model)

        global time_end
        #otherX y OtherY es la posicion del semaforo al que va a estar viendo. todos en un carril pueden ver a solo uno de otro carril porque seguiran el mismo patron por carril.

        #El semaforo va a tener 3 estados en su live. Estado 1 es rojo, estado 2 es amarillo, estado 3 es verde. El estado amarillo no afecta a otros semaforos, pero si a los carros, cuando es amarillo, estos paran su movimiento.
        #Cuando el semaforo esta en rojo, va a ser detectado por los semaforos de los carriles que cruzen a los actuales y estos cambiaran a verde.
        #El semaforo estara verde por una cierta cantidad de tiempo, igual para el estado amarillo. El rojo cambiara solo cuando un semaforo observado este en verde, por lo que permanecera en rojo el tiempo que
        # el otro semaforo este en verde + amarillo.

        #nosotros declaramos que linea empieza verde primero en el modelo.


        #si es primero
        if isFirst == True:
            #verde
            self.live = 3
        else:
            #si no, rojo
            self.live = 1
        
        
        #su siguiente estado es si mismo
        #self.next_state = self.live
        

    def step(self):
        this_cell = self.model.grid.get_cell_list_contents([(otherX,otherY)])
        for content in this_cell:

          #si el content es otro semaforo
          if isinstance(content, Semaforo):

            #(este rojo y el otro rojo, cambia a verde.)
            if (self.live == 1 and content.live == 1):
                self.next_state = 3

                # (hace un tiempo verde, luego amarillo, luego rojo)
            elif self.live == 3:
                
                #si es verde, y el tiempo es mayor a time_end, cambia a amarillo y agrega el tiempo de espera a amarillo.
                if time.time() > time_end:
                  time_end = time.time() + 5
                  self.next_state = 2
            else:
                #cambiamos de amarillo a rojo, como el tiempo end es global, lo cambiamos a el timer del verde.
                if time.time() > time_end:
                  time_end = time.time() + 5
                  self.next_state = 1
            

    def advance(self):
        self.live = self.next_state

In [ ]:
#agente carro
class Carro(Agent):

    def __init__(self, unique_id, pos, model, direccion, posDestino):

        super().__init__(unique_id, model)

        #si es primero
        self.live = 5
        self.x,self.y = pos
        self.xD, self.yD = posDestino
        self.canMove = True
        self.willTurn = random.randint(0, 1)
        self.revisa = True

        if direccion == -1 or direccion == 1:
          #HV = 0 = horizontal, HV = 1 = vertical
          self.HV == 1
        else:
          self.HV == 0

    def step(self):

        #la direccion hay que darla por ints

        #   -1  =  abajo
        #    1  =  arriba
        #    2  =  derecha
        #   -2  =  izquierda
        

        this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y)])
        if isinstance(content,Semaforo):
                if content.live == 1 or content.live == 2:
                  self.canMove = False
                else:
                  self.canMove = True
                  self.revisaNext = False

        #MOVIMIENTO EN LAS LINEAS VERTICALES
        if HV == 1 and canMove == True:
          #self y - 2

          if direccion == -1:
            if revisa == True:
              this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y-1)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True
              this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y-2)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True

          elif direccion == 1:
            if revisa == True:
              this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y+1)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True
              this_cell = self.model.grid.get_cell_list_contents([(self.x,self.y+2)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True

          if canMove == True:
            if self.y < self.yD:
              self.model.grid.move_agent(self,(self.x,self.y+1))
            elif self.y > self.yD:
              self.model.grid.move_agent(self,(self.x,self.y+1))
            elif self.x < self.xD:
              self.model.grid.move_agent(self,(self.x+1,self.y))
            elif self.x > self.xD:
              self.model.grid.move_agent(self,(self.x-1,self.y))

        #MOVIMIENTO EN LAS LINEAS HORIZONTALES
        if HV == 0 and canMove == True:
          #self y - 2

          if direccion == -2:
            if revisa == True:
              this_cell = self.model.grid.get_cell_list_contents([(self.x-1,self.y)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True
              this_cell = self.model.grid.get_cell_list_contents([(self.x-2,self.y)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True

          elif direccion == 2:
            if revisa == True:
              this_cell = self.model.grid.get_cell_list_contents([(self.x+1,self.y)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True
              this_cell = self.model.grid.get_cell_list_contents([(self.x+2,self.y)])
              for content in this_cell:
                  if isinstance(content,Carro):
                    self.canMove = False
                  else:
                    self.canMove = True

          if canMove == True:
            if self.x < self.xD:
              self.model.grid.move_agent(self,(self.x+1,self.y))
            elif self.x > self.xD:
              self.model.grid.move_agent(self,(self.x-1,self.y))
            elif self.y < self.yD:
              self.model.grid.move_agent(self,(self.x,self.y+1))
            elif self.y > self.yD:
              self.model.grid.move_agent(self,(self.x,self.y+1))


    def advance(self):
        self.live = self.next_state
        self.revisa = self.revisaNext

##Modelo

In [ ]:
from mesa import Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from queue import Queue


class RobotBoxes(Model):
    def __init__(self, width, height):
        self.semaforos=4
        self.carros=4
        self.grid = MultiGrid(width, height, False)
        self.schedule = SimultaneousActivation(self)
        q = Queue(maxsize = 4)

        # Sapwn Carros
        carro1 = Carro(1, (width/2,0), self, 2, (4,9))
        self.grid.place_agent(carro1, (0,5))
        self.schedule.add(carro1)

        carro2 = Carro(2, (width-1,height/2-1), self, 2, (4,9))
        self.grid.place_agent(carro2, (4,0))
        self.schedule.add(carro2)

        carro3 = Carro(3, (width/2,height-1), self, 2, (9,5))
        self.grid.place_agent(carro3, (9,4))
        self.schedule.add(carro3)

        carro4 = Carro(4, (0,height/2), self, 2, (4,9))
        self.grid.place_agent(carro4, (5,9))
        self.schedule.add(carro4)

        #Spawn Semaforos
        semaforo1 = Semaforo(5, (x/2-1,y/2-2), self, 2, (4,9))
        self.grid.place_agent(semaforo1, (0,5))
        self.schedule.add(semaforo1)

        semaforo2 = Semaforo(6, (x/2+2,y/2-1), self, 2, (4,9))
        self.grid.place_agent(semaforo2, (4,0))
        self.schedule.add(semaforo2)

        semaforo3 = Semaforo(7, (x/2,y/2+2), self, 2, (9,5))
        self.grid.place_agent(semaforo3, (9,4))
        self.schedule.add(semaforo3)

        semaforo4 = Semaforo(8, (x/2-2,y/2), self, 2, (4,9))
        self.grid.place_agent(semaforo4, (5,9))
        self.schedule.add(semaforo4)

        '''
        Debe detectar cuando un carro spawnea en la calle para entrar al queue
        si no hay carro en esa calle se hace un pop de la misma.
        '''
        
        q.put(semaforo1)
        q.put(semaforo2)
        q.put(semaforo3)
        q.put(semaforo4)

        '''
        Aquí pondremos el algoritmo para el programa de luces, falta implementarlo
        con el queue para obtener un orden de los semaforos
        '''
        # Aquí definimos con colector para obtener el grid completo.
        self.datacollector = DataCollector(
            model_reporters={"Grid": get_grid})
        
    def step(self):
        '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
        '''
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
#Datos Iniciales
tiempoEjecucion = 1.0
iniRobots = 5
iniBoxes = int(input("Ingresa numero de cajas: "))
width = 10 #Ancho predeterminado
height = 10 #alto predeterminado
NUM_GENERATIONS = 0

start_time = time.time()
tiempo_inicio=str(datetime.timedelta(seconds=tiempoEjecucion))
model = RobotBoxes(width, height, iniBoxes, iniRobots)
while((time.time()-start_time) < tiempoEjecucion and boxesReady < iniBoxes and NUM_GENERATIONS <= 250):
    model.step()
    NUM_GENERATIONS += 1


print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))
print('Numero total de movimiento: ', TOTAL_MOVES)
if(NUM_GENERATIONS > 250):
  print("No se apilaron todas las cajas")
else:
  print("Se apilaron todas las cajas")

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
print(type(all_grid))
print(all_grid)

In [ ]:
anim